## Link to a Google Drive where data will be held

First must attain credentials to access google drive. 

In [63]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import os
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [64]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=634826925860-6p7atcthpng7p74qts4q8dceissp6m20.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [65]:
folder = "SpotifyScrapedData"
folder_present = False
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
    if file1['title'] == folder:
        folder_present = True
        id = file1['id']
        
if not folder_present:
    file1 = drive.CreateFile({'title': folder, 
    "mimeType": "application/vnd.google-apps.folder"})
    file1.Upload()
    id = file1['id']
    
    
folder_file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(id)}).GetList()
for file1 in folder_file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))
title = "mySpotifyListening.csv"
file_present = False
for file2 in folder_file_list:
    if file2["title"] == title:
        file_present = True
        id = file1['id']

if not file_present: 
    file1 = drive.CreateFile({'title': title, 
                              "parents": [{"id": id}], 
    "mimeType": "csv"})
    file1.Upload()

title: mySpotifyListening.csv, id: 1BP2mtfRbW4XihfYehfIBJQEeLy9ouOtD


In [ ]:
clientId = os.environ.get('spotifyClientId')
clientSecret = os.environ.get('spotifyClientSecret')
clientCredentialsManager = SpotifyClientCredentials(client_id=clientId, client_secret=clientSecret)
sp = spotipy.Spotify(client_credentials_manager=clientCredentialsManager) #spotify object to access API

spotifyUsername =os.environ.get("spotifyUsername")
scope = 'user-top-read user-read-recently-played'
SPOTIPY_REDIRECT_URI=os.environ.get('redirect_uri')
token = util.prompt_for_user_token(spotifyUsername, scope, clientId, clientSecret,redirect_uri=SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)
